In [61]:
import pandas as pd
import os
import ast, json
import math

In [127]:
dff=pd.read_csv(r"E:\KreupAI\DAta\Event\23-24\Almeria vs Athletic Club.csv")
dff.head()

,Unnamed: 0,id,eventId,minute,second,teamId,x,y,expandedMinute,period,...,isShot,cardType,teamName,oppositionTeamName,shirtNo,name,position,isFirstEleven,ishome,datetime
0,0,2.645844e+09,2,0,0.0,53,0.0,0.0,0,1,...,NaN,NaN,Athletic Club,Almeria,NaN,NaN,NaN,NaN,0,2024-02-13 06:33:43
1,1,2.645844e+09,2,0,0.0,1799,0.0,0.0,0,1,...,NaN,NaN,Almeria,Athletic Club,NaN,NaN,NaN,NaN,1,2024-02-13 06:33:43
2,2,2.645844e+09,3,0,0.0,53,50.2,49.8,0,1,...,NaN,NaN,Athletic Club,Almeria,20.0,Asier Villalibre,FW,True,0,2024-02-13 06:33:43
3,3,2.645844e+09,4,0,4.0,53,54.8,48.6,0,1,...,NaN,NaN,Athletic Club,Almeria,30.0,Unai Gomez,AMC,True,0,2024-02-13 06:33:43
4,4,2.645844e+09,5,0,7.0,53,46.5,84.7,0,1,...,NaN,NaN,Athletic Club,Almeria,17.0,Yuri Berchiche,DL,True,0,2024-02-13 06:33:43


In [129]:
for i in dff.columns:
    print(i)

Unnamed: 0
id
eventId
minute
second
teamId
x
y
expandedMinute
period
type
outcomeType
qualifiers
satisfiedEventsTypes
isTouch
playerId
endX
endY
relatedEventId
relatedPlayerId
blockedX
blockedY
goalMouthZ
goalMouthY
isShot
cardType
teamName
oppositionTeamName
shirtNo
name
position
isFirstEleven
ishome
datetime


In [139]:
ctk=['type','outcomeType','isGoal','isOwnGoal','isShot','ishome','teamName','oppositionTeamName','datetime']
for col in ctk:
    if col not in dff.columns:
        dff[col] = False

df = dff[ctk]

In [141]:
df

,type,outcomeType,isGoal,isOwnGoal,isShot,ishome,teamName,oppositionTeamName
0,Start,Successful,False,False,NaN,0,Athletic Club,Almeria
1,Start,Successful,False,False,NaN,1,Almeria,Athletic Club
2,Pass,Successful,False,False,NaN,0,Athletic Club,Almeria
3,Pass,Successful,False,False,NaN,0,Athletic Club,Almeria
4,Pass,Successful,False,False,NaN,0,Athletic Club,Almeria
...,...,...,...,...,...,...,...,...
1541,End,Successful,False,False,NaN,0,Athletic Club,Almeria
1542,End,Successful,False,False,NaN,0,Athletic Club,Almeria
1543,End,Successful,False,False,NaN,1,Almeria,Athletic Club
1544,FormationSet,Successful,False,False,NaN,1,Almeria,Athletic Club


In [175]:
import pandas as pd
import numpy as np

def compute_goal_stats(df, match_col=None):
    """
    Compute goals scored, conceded, opponent, and result (W/D/L) per team.
    
    Parameters
    ----------
    df : pd.DataFrame
        Must contain: teamName, oppositionTeamName, ishome, isGoal, isOwnGoal.
    match_col : str or None
        Optional column to group by match ID.
        
    Returns
    -------
    result : pd.DataFrame
        Columns: matchId?, team, opponent, ishome, goals_scored, goals_conceded, result(W/D/L)
    pivot : pd.DataFrame
        Pivoted summary.
    """
    def truthy(x):
        if pd.isna(x): return False
        if isinstance(x, (bool, np.bool_)): return bool(x)
        if isinstance(x, (int, np.integer, float, np.floating)): return bool(x)
        return str(x).strip().lower() in {"1","true","t","yes","y"}

    def to_int_home(x):
        if pd.isna(x): return np.nan
        try:
            return int(float(x))
        except Exception:
            return np.nan

    df = df.copy()
    df["isGoal_bool"] = df["isGoal"].apply(truthy)
    df["isOwnGoal_bool"] = df["isOwnGoal"].apply(truthy)
    df["ishome_int"] = df["ishome"].apply(to_int_home)

    goals = df[df["isGoal_bool"]].copy()
    if goals.empty:
        return pd.DataFrame(), pd.DataFrame()

    # Assign scorer and conceder details
    goals["scoring_team"] = np.where(goals["isOwnGoal_bool"], goals["oppositionTeamName"], goals["teamName"])
    goals["scoring_home"] = np.where(goals["isOwnGoal_bool"], 1 - goals["ishome_int"], goals["ishome_int"])
    goals["scoring_opponent"] = np.where(goals["isOwnGoal_bool"], goals["teamName"], goals["oppositionTeamName"])

    goals["conceding_team"] = np.where(goals["isOwnGoal_bool"], goals["teamName"], goals["oppositionTeamName"])
    goals["conceding_home"] = np.where(goals["isOwnGoal_bool"], goals["ishome_int"], 1 - goals["ishome_int"])
    goals["conceding_opponent"] = np.where(goals["isOwnGoal_bool"], goals["oppositionTeamName"], goals["teamName"])

    rows = []
    for _, r in goals.iterrows():
        keys = {}
        if match_col and match_col in r:
            keys["matchId"] = r[match_col]

        rows.append({**keys,
            "team": r["scoring_team"],
            "opponent": r["scoring_opponent"],
            "ishome": int(r["scoring_home"]) if not pd.isna(r["scoring_home"]) else -1,
            "goals_scored": 1, "goals_conceded": 0})
        rows.append({**keys,
            "team": r["conceding_team"],
            "opponent": r["conceding_opponent"],
            "ishome": int(r["conceding_home"]) if not pd.isna(r["conceding_home"]) else -1,
            "goals_scored": 0, "goals_conceded": 1})

    tidy = pd.DataFrame(rows)
    group_cols = ["team","opponent","ishome",",'datetime']"]
    if match_col:
        group_cols.insert(0,"matchId")

    result = tidy.groupby(group_cols, dropna=False)[["goals_scored","goals_conceded"]].sum().reset_index()

    # Compute W/D/L
    def compute_result(gs,gc):
        if gs > gc: return "W"
        if gs < gc: return "L"
        return "D"

    result["result"] = result.apply(lambda x: compute_result(x["goals_scored"], x["goals_conceded"]), axis=1)


    return result


In [137]:
# df = your events dataframe
result= compute_goal_stats(df)


result



(Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [])

In [193]:
import pandas as pd
import numpy as np
import os

# === CONFIG ===
dir_path = r"E:\KreupAI\DAta\Event\23-24"
ctk = ['type','outcomeType','isGoal','isOwnGoal','isShot','ishome','teamName','oppositionTeamName','datetime']

def truthy(x):
    if pd.isna(x): return False
    if isinstance(x, (bool, np.bool_)): return bool(x)
    if isinstance(x, (int, np.integer, float, np.floating)): return bool(x)
    return str(x).strip().lower() in {"1","true","t","yes","y"}

def to_int_home(x):
    if pd.isna(x): return np.nan
    try:
        return int(float(x))
    except Exception:
        return np.nan

def compute_goal_stats(df, matchname):
    
    df = df.copy()
    df["isGoal_bool"] = df["isGoal"].apply(truthy)
    df["isOwnGoal_bool"] = df["isOwnGoal"].apply(truthy)
    df["ishome_int"] = df["ishome"].apply(to_int_home)

    # Identify the teams involved in this match
    # Use unique pairs (team, opponent, ishome) from all rows
    unique_pairs = df.dropna(subset=["teamName","oppositionTeamName","ishome"])\
                     [["teamName","oppositionTeamName","ishome"]].drop_duplicates()

    goals = df[df["isGoal_bool"]].copy()

    rows = []
    if goals.empty:
        # No goals: create zero-goal rows for both teams
        for _, row in unique_pairs.iterrows():
            rows.append({
                "matchname": matchname,
                "team": row["teamName"],
                "opponent": row["oppositionTeamName"],
                "ishome": int(row["ishome"]),
                "goals_scored": 0,
                "goals_conceded": 0,
                "result": "D"
            })
        return pd.DataFrame(rows)

    # Process goals normally
    goals["scoring_team"] = np.where(goals["isOwnGoal_bool"], goals["oppositionTeamName"], goals["teamName"])
    goals["scoring_home"] = np.where(goals["isOwnGoal_bool"], 1 - goals["ishome_int"], goals["ishome_int"])
    goals["scoring_opponent"] = np.where(goals["isOwnGoal_bool"], goals["teamName"], goals["oppositionTeamName"])

    goals["conceding_team"] = np.where(goals["isOwnGoal_bool"], goals["teamName"], goals["oppositionTeamName"])
    goals["conceding_home"] = np.where(goals["isOwnGoal_bool"], goals["ishome_int"], 1 - goals["ishome_int"])
    goals["conceding_opponent"] = np.where(goals["isOwnGoal_bool"], goals["oppositionTeamName"], goals["teamName"])

    for _, r in goals.iterrows():
        rows.append({
            "matchname": matchname,
            "team": r["scoring_team"],
            "opponent": r["scoring_opponent"],
            "ishome": int(r["scoring_home"]) if not pd.isna(r["scoring_home"]) else -1,
            "goals_scored": 1, "goals_conceded": 0
        })
        rows.append({
            "matchname": matchname,
            "team": r["conceding_team"],
            "opponent": r["conceding_opponent"],
            "ishome": int(r["conceding_home"]) if not pd.isna(r["conceding_home"]) else -1,
            "goals_scored": 0, "goals_conceded": 1
        })

    tidy = pd.DataFrame(rows)

    result = tidy.groupby(["matchname","team","opponent","ishome"], dropna=False)\
                 [["goals_scored","goals_conceded"]].sum().reset_index()

    def compute_result(gs, gc):
        if gs > gc: return "W"
        if gs < gc: return "L"
        return "D"

    result["result"] = result.apply(lambda x: compute_result(x["goals_scored"], x["goals_conceded"]), axis=1)
    return result

# === MAIN LOOP ===
all_results = []
all_matches = []

for file in os.listdir(dir_path):
    if file.endswith(".csv"):
        matchname = os.path.splitext(file)[0]
        all_matches.append(matchname)

        file_path = os.path.join(dir_path, file)
        dff = pd.read_csv(file_path)

        # Ensure required columns exist
        for col in ctk:
            if col not in dff.columns:
                dff[col] = False

        df = dff[ctk]
        res = compute_goal_stats(df, matchname)
        all_results.append(res)


final_df = pd.concat(all_results, ignore_index=True) if all_results else pd.DataFrame()
final_df['season']='2023-24'

print(final_df.head())
print(f"Total matches processed (including 0-0 draws): {final_df['matchname'].nunique()}")


                  matchname           team       opponent  ishome  \
0  Almeria vs Athletic Club  Athletic Club        Almeria       0   
1  Almeria vs Athletic Club        Almeria  Athletic Club       1   
2       Almeria vs Atletico        Almeria       Atletico       1   
3       Almeria vs Atletico       Atletico        Almeria       0   
4      Almeria vs Barcelona        Almeria      Barcelona       1   

   goals_scored  goals_conceded result   season  
0             0               0      D  2023-24  
1             0               0      D  2023-24  
2             2               2      D  2023-24  
3             2               2      D  2023-24  
4             0               2      L  2023-24  
Total matches processed (including 0-0 draws): 380


In [194]:
final_df

,matchname,team,opponent,ishome,goals_scored,goals_conceded,result,season
0,Almeria vs Athletic Club,Athletic Club,Almeria,0,0,0,D,2023-24
1,Almeria vs Athletic Club,Almeria,Athletic Club,1,0,0,D,2023-24
2,Almeria vs Atletico,Almeria,Atletico,1,2,2,D,2023-24
3,Almeria vs Atletico,Atletico,Almeria,0,2,2,D,2023-24
4,Almeria vs Barcelona,Almeria,Barcelona,1,0,2,L,2023-24
...,...,...,...,...,...,...,...,...
755,Villarreal vs Real Sociedad,Villarreal,Real Sociedad,1,0,3,L,2023-24
756,Villarreal vs Sevilla,Sevilla,Villarreal,0,2,3,L,2023-24
757,Villarreal vs Sevilla,Villarreal,Sevilla,1,3,2,W,2023-24
758,Villarreal vs Valencia,Valencia,Villarreal,0,0,1,L,2023-24


In [195]:
import pandas as pd
import numpy as np
import os

# === CONFIG ===
dir_path = r"E:\KreupAI\DAta\Event\24-25"
ctk = ['type','outcomeType','isGoal','isOwnGoal','isShot','ishome','teamName','oppositionTeamName']

def truthy(x):
    if pd.isna(x): return False
    if isinstance(x, (bool, np.bool_)): return bool(x)
    if isinstance(x, (int, np.integer, float, np.floating)): return bool(x)
    return str(x).strip().lower() in {"1","true","t","yes","y"}

def to_int_home(x):
    if pd.isna(x): return np.nan
    try:
        return int(float(x))
    except Exception:
        return np.nan

def compute_goal_stats(df, matchname):
    df = df.copy()
    df["isGoal_bool"] = df["isGoal"].apply(truthy)
    df["isOwnGoal_bool"] = df["isOwnGoal"].apply(truthy)
    df["ishome_int"] = df["ishome"].apply(to_int_home)

    # Identify the teams involved in this match
    # Use unique pairs (team, opponent, ishome) from all rows
    unique_pairs = df.dropna(subset=["teamName","oppositionTeamName","ishome"])\
                     [["teamName","oppositionTeamName","ishome"]].drop_duplicates()

    goals = df[df["isGoal_bool"]].copy()

    rows = []
    if goals.empty:
        # No goals: create zero-goal rows for both teams
        for _, row in unique_pairs.iterrows():
            rows.append({
                "matchname": matchname,
                "team": row["teamName"],
                "opponent": row["oppositionTeamName"],
                "ishome": int(row["ishome"]),
                "goals_scored": 0,
                "goals_conceded": 0,
                "result": "D"
            })
        return pd.DataFrame(rows)

    # Process goals normally
    goals["scoring_team"] = np.where(goals["isOwnGoal_bool"], goals["oppositionTeamName"], goals["teamName"])
    goals["scoring_home"] = np.where(goals["isOwnGoal_bool"], 1 - goals["ishome_int"], goals["ishome_int"])
    goals["scoring_opponent"] = np.where(goals["isOwnGoal_bool"], goals["teamName"], goals["oppositionTeamName"])

    goals["conceding_team"] = np.where(goals["isOwnGoal_bool"], goals["teamName"], goals["oppositionTeamName"])
    goals["conceding_home"] = np.where(goals["isOwnGoal_bool"], goals["ishome_int"], 1 - goals["ishome_int"])
    goals["conceding_opponent"] = np.where(goals["isOwnGoal_bool"], goals["oppositionTeamName"], goals["teamName"])

    for _, r in goals.iterrows():
        rows.append({
            "matchname": matchname,
            "team": r["scoring_team"],
            "opponent": r["scoring_opponent"],
            "ishome": int(r["scoring_home"]) if not pd.isna(r["scoring_home"]) else -1,
            "goals_scored": 1, "goals_conceded": 0
        })
        rows.append({
            "matchname": matchname,
            "team": r["conceding_team"],
            "opponent": r["conceding_opponent"],
            "ishome": int(r["conceding_home"]) if not pd.isna(r["conceding_home"]) else -1,
            "goals_scored": 0, "goals_conceded": 1
        })

    tidy = pd.DataFrame(rows)

    result = tidy.groupby(["matchname","team","opponent","ishome"], dropna=False)\
                 [["goals_scored","goals_conceded"]].sum().reset_index()

    def compute_result(gs, gc):
        if gs > gc: return "W"
        if gs < gc: return "L"
        return "D"

    result["result"] = result.apply(lambda x: compute_result(x["goals_scored"], x["goals_conceded"]), axis=1)

    return result

# === MAIN LOOP ===
all_results = []
all_matches = []

for file in os.listdir(dir_path):
    if file.endswith(".csv"):
        matchname = os.path.splitext(file)[0]
        all_matches.append(matchname)

        file_path = os.path.join(dir_path, file)
        dff = pd.read_csv(file_path)

        # Ensure required columns exist
        for col in ctk:
            if col not in dff.columns:
                dff[col] = False

        df = dff[ctk]
        res = compute_goal_stats(df, matchname)
        all_results.append(res)

# Concatenate
final_df1 = pd.concat(all_results, ignore_index=True) if all_results else pd.DataFrame()
final_df1['season']='2024-25'

print(final_df1.head())
print(f"Total matches processed (including 0-0 draws): {final_df['matchname'].nunique()}")


                     matchname           team       opponent  ishome  \
0    Athletic Club vs Atletico  Athletic Club       Atletico       1   
1    Athletic Club vs Atletico       Atletico  Athletic Club       0   
2   Athletic Club vs Barcelona  Athletic Club      Barcelona       1   
3   Athletic Club vs Barcelona      Barcelona  Athletic Club       0   
4  Athletic Club vs Celta Vigo  Athletic Club     Celta Vigo       1   

   goals_scored  goals_conceded result   season  
0             0               1      L  2024-25  
1             1               0      W  2024-25  
2             0               3      L  2024-25  
3             3               0      W  2024-25  
4             3               1      W  2024-25  
Total matches processed (including 0-0 draws): 380


In [196]:
final_df1

,matchname,team,opponent,ishome,goals_scored,goals_conceded,result,season
0,Athletic Club vs Atletico,Athletic Club,Atletico,1,0,1,L,2024-25
1,Athletic Club vs Atletico,Atletico,Athletic Club,0,1,0,W,2024-25
2,Athletic Club vs Barcelona,Athletic Club,Barcelona,1,0,3,L,2024-25
3,Athletic Club vs Barcelona,Barcelona,Athletic Club,0,3,0,W,2024-25
4,Athletic Club vs Celta Vigo,Athletic Club,Celta Vigo,1,3,1,W,2024-25
...,...,...,...,...,...,...,...,...
755,Villarreal vs Real Valladolid,Villarreal,Real Valladolid,1,5,1,W,2024-25
756,Villarreal vs Sevilla,Sevilla,Villarreal,0,2,4,L,2024-25
757,Villarreal vs Sevilla,Villarreal,Sevilla,1,4,2,W,2024-25
758,Villarreal vs Valencia,Valencia,Villarreal,0,1,1,D,2024-25


In [205]:
merged_df = pd.concat([final_df, final_df1], ignore_index=True)
merged_df.to_csv("E:/KreupAI/DAta/results.csv")